In [ ]:
#文件夹中图像降噪
import cv2
import numpy as np
import os

def remove_noise(image_array):
  kernel_size = 3

  # Iterate through each pixel
  for y in range(image_array.shape[0]):
    for x in range(image_array.shape[1]):
      if image_array[y, x] > 0:
        value_region = image_array[y-kernel_size//2:y+kernel_size//2+1, x-kernel_size//2:x+kernel_size//2+1]
        value_sum = np.sum(value_region)
        if value_sum < 50:
          image_array[y-kernel_size//2:y+kernel_size//2+1, x-kernel_size//2:x+kernel_size//2+1] = 0

  return image_array
input_folder = "E:\\data\\12after\\after\\NOR20"
output_folder = "E:\\data\\12after\\denoise\\NOR20"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
  if filename.endswith(".png") or filename.endswith(".jpg"):
    image = cv2.imread(os.path.join(input_folder, filename), cv2.IMREAD_GRAYSCALE)
    image_array = image.astype(np.uint8)
    denoised_array = remove_noise(image_array.copy())
    denoised_image = denoised_array.astype(np.uint8)
    denoised_image = cv2.cvtColor(denoised_image, cv2.COLOR_GRAY2BGR)
    cv2.imwrite(os.path.join(output_folder, filename), denoised_image)

print("Noise reduction and image processing complete!")
print(f"Denoised images saved to {output_folder}")


In [ ]:
#文件夹中图像增强
import cv2
import numpy as np
import os
from scipy.interpolate import RectBivariateSpline

def bicubic_interpolation(box):
    x = np.arange(0, box.shape[1])
    y = np.arange(0, box.shape[0])
    new_func = RectBivariateSpline(y, x, box)

    xnew = np.linspace(0, box.shape[1], 9)
    ynew = np.linspace(0, box.shape[0], 9)
    znew = new_func(ynew, xnew)

    return znew

def process_image(image):
    sigma_s = 5
    sigma_r = 30
    region_size = 5

    for i in range(0, image.shape[0], 5):
        for j in range(0, image.shape[1], 5):
            box = image[i:i+5, j:j+5]

            if 255 in box:
                continue
            else:
                interpolated_box = bicubic_interpolation(box)
                interpolated_box = cv2.resize(interpolated_box, (5, 5))
                image[i:i+5, j:j+5] = interpolated_box
            if 255 in box and len(np.unique(box)) > 1:
                box = cv2.bilateralFilter(box, region_size, sigma_s, sigma_r)

    return image

# Directory containing the images
input_dir = "E:\\data\\12after\\denoise\\NOR20"
output_dir = "E:\\data\\12after\\enhanced\\NOR20"

# Process all images in the directory
for filename in os.listdir(input_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        image = cv2.imread(os.path.join(input_dir, filename), cv2.IMREAD_GRAYSCALE)
        processed_image = process_image(image)
        cv2.imwrite(os.path.join(output_dir, filename), processed_image)
